<a href="https://colab.research.google.com/github/peppe-sc/AML_project_poliTO/blob/test_GS/AML_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!jar xvf  "/content/drive/MyDrive/AML/Cityscapes.zip"

In [2]:
#!jar xvf  "/content/drive/MyDrive/AML/GTA5.zip"
!pip install tensorboardX

In [3]:
import os
import os.path

if not os.path.isdir('/content/AML_project_poliTO'):
  !git clone -b test_GS https://github.com/peppe-sc/AML_project_poliTO.git
  %cd AML_project_poliTO

if os.path.isdir('./AML_project_poliTO'):
  %cd AML_project_poliTO


!git pull origin test_GS



/content/AML_project_poliTO
From https://github.com/peppe-sc/AML_project_poliTO
 * branch            test_GS    -> FETCH_HEAD
Already up to date.


In [4]:
optimizer = "adam"
num_epochs = 50
learning_rate = 0.01
batch_size = 5
num_classes = 19
checkpoint_step = 1
save_model_path = "/content/my_checkpoint"
validation_step = 10
mode = "train"
num_workers = 2
backbone = "STDCNet813"
pretrain_path ="checkpoints/STDCNet813M_73.91.tar" #"checkpoints/latest.pth" #"checkpoints/STDCNet813M_73.91.tar"
use_conv_last = False
use_gpu = True
n_classes = num_classes


start_epoch = 0
for check in os.listdir('/content/AML_project_poliTO/checkpoints'):
  if 'latest_' in check:

    start_epoch_tmp = int(check.split('_')[1].replace('.pth',''))

    if start_epoch_tmp >= start_epoch:
      start_epoch = start_epoch_tmp+1
      pretrain_path = "checkpoints/"+check


print("Starting at Epoch: " + str(start_epoch))

Starting at Epoch: 2


In [6]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.transforms import v2
from torchvision.models import alexnet

from PIL import Image
from tqdm import tqdm

In [7]:
from model.model_stages import BiSeNet
from cityscapes import CityScapes
from gta5 import GTA5
import torch
from torch.utils.data import DataLoader
import logging
import argparse
import numpy as np

from tensorboardX import SummaryWriter
import torch.cuda.amp as amp
from utils import poly_lr_scheduler
from utils import reverse_one_hot, compute_global_accuracy, fast_hist, per_class_iu
from tqdm import tqdm

logger = logging.getLogger()


def val(model, dataloader):
    print('start val!')
    with torch.no_grad():
        model.eval()
        precision_record = []
        hist = np.zeros((num_classes, num_classes))
        for i, (data, label) in enumerate(dataloader):
            label = label.type(torch.LongTensor)
            data = data.cuda()
            label = label.long().cuda()

            # get RGB predict image
            predict, _, _ = model(data)
            predict = predict.squeeze(0)
            predict = reverse_one_hot(predict)
            predict = np.array(predict.cpu())

            # get RGB label image
            label = label.squeeze()
            label = np.array(label.cpu())

            # compute per pixel accuracy
            precision = compute_global_accuracy(predict, label)
            hist += fast_hist(label.flatten(), predict.flatten(), num_classes)

            # there is no need to transform the one-hot array to visual RGB array
            # predict = colour_code_segmentation(np.array(predict), label_info)
            # label = colour_code_segmentation(np.array(label), label_info)
            precision_record.append(precision)

        precision = np.mean(precision_record)
        miou_list = per_class_iu(hist)
        miou = np.mean(miou_list)
        print('precision per pixel for test: %.3f' % precision)
        print('mIoU for validation: %.3f' % miou)
        print(f'mIoU per class: {miou_list}')

        return precision, miou

def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Unsupported value encountered.')

In [8]:
import math
def train( model, optimizer, dataloader_train, dataloader_val):
    writer = SummaryWriter(comment=''.format(optimizer))
    nans = []
    scaler = amp.GradScaler()

    loss_func = torch.nn.CrossEntropyLoss(ignore_index=255)
    max_miou = 0
    step = start_epoch #0
    for epoch in range(start_epoch,num_epochs):
        lr = poly_lr_scheduler(optimizer, learning_rate, iter=epoch, max_iter=num_epochs)
        model.train()
        tq = tqdm(total=len(dataloader_train) * batch_size)
        tq.set_description('epoch %d, lr %f' % (epoch, lr))
        loss_record = []
        for i, (data, label) in enumerate(dataloader_train):
            data = data.cuda()
            label = label.long().cuda()
            optimizer.zero_grad()

           #print("\n")
           #print(data.size())
           #print("\n")
           #print(label.size())
           #print(torch.squeeze(label,1).size())
           #print("\n")

            with amp.autocast():
                output, out16, out32 = model(data)
                #print("###")
                #print(label.squeeze(1).size())
                #print("###")
                #print(out16.size())
                #print("###")
                #print(out32.size())
                loss1 = loss_func(output, label.squeeze(1))
                loss2 = loss_func(out16, label.squeeze(1))
                loss3 = loss_func(out32, label.squeeze(1))
                loss = loss1 + loss2 + loss3
                #print(loss)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            tq.update(batch_size)
            tq.set_postfix(loss='%.6f' % loss)
            step += 1
            writer.add_scalar('loss_step', loss, step)
            loss_record.append(loss.item())
            if math.isnan(loss.item()):
              nans.append(loss.item())
        tq.close()
        loss_train_mean = np.mean(loss_record)
        writer.add_scalar('epoch/loss_epoch_train', float(loss_train_mean), epoch)
        print('loss for train : %f' % (loss_train_mean))
        if epoch % checkpoint_step == 0 and epoch != 0:
            #print("\n")
            #print(nans)
            #print("\n")
            import os
            if not os.path.isdir(save_model_path):
                os.mkdir(save_model_path)
            torch.save({'state_dict':model.module.state_dict(),'optimizer_state_dict': optimizer.state_dict()}, os.path.join(save_model_path, 'latest_'+str(epoch)+'.pth'))

        if epoch % validation_step == 0 and epoch != 0:
            precision, miou = val( model, dataloader_val)
            if miou > max_miou:
                max_miou = miou
                import os
                os.makedirs(save_model_path, exist_ok=True)
                torch.save(model.module.state_dict(), os.path.join(save_model_path, 'best.pth'))
            writer.add_scalar('epoch/precision_val', precision, epoch)
            writer.add_scalar('epoch/miou val', miou, epoch)


In [9]:
train_dataset = CityScapes(mode='train',rootpth='/content/Cityscapes/Cityspaces')
dataloader_train = DataLoader(train_dataset,
                batch_size=batch_size,
                shuffle=False,
                num_workers=num_workers,
                pin_memory=False,
                drop_last=True)

val_dataset = CityScapes(mode='val',rootpth='/content/Cityscapes/Cityspaces')
dataloader_val = DataLoader(val_dataset,
                   batch_size=1,
                   shuffle=False,
                   num_workers=num_workers,
                   drop_last=False)

self.mode train
self.len train 1572
self.mode val
self.len val 500


In [8]:
train_dataset = GTA5(mode='train',rootpth='/content/Cityscapes/GTA5')
dataloader_train = DataLoader(train_dataset,
                batch_size=batch_size,
                shuffle=False,
                num_workers=num_workers,
                pin_memory=False,
                drop_last=True)

val_dataset = GTA5(mode='val',rootpth='/content/Cityscapes/GTA5')
dataloader_val = DataLoader(val_dataset,
                   batch_size=1,
                   shuffle=False,
                   num_workers=num_workers,
                   drop_last=False)

self.mode train


FileNotFoundError: ignored

In [11]:
## model
#print(torch.load(pretrain_path))
model = BiSeNet(backbone=backbone, n_classes=n_classes, pretrain_model="checkpoints/STDCNet813M_73.91.tar", use_conv_last=use_conv_last)

#if pretrain_path != "checkpoints/STDCNet813M_73.91.tar":
#  checkpoint = torch.load(pretrain_path)
#  model.load_state_dict(checkpoint['state_dict'])

if torch.cuda.is_available() and use_gpu:
    model = torch.nn.DataParallel(model).cuda()

## optimizer
# build optimizer
print(optimizer)
if optimizer == 'rmsprop':
    optimizer = torch.optim.RMSprop(model.parameters(), learning_rate)
elif optimizer == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), learning_rate, momentum=0.9, weight_decay=1e-4)
elif optimizer == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), learning_rate)
else:  # rmsprop
    print('not supported optimizer, using adam\n')
    optimizer = torch.optim.Adam(model.parameters(), learning_rate)

if pretrain_path != "checkpoints/STDCNet813M_73.91.tar":

  checkpoint = torch.load(pretrain_path)
  model.module.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
## train loop

train(model, optimizer, dataloader_train, dataloader_val)
# final test
val(model, dataloader_val)

use pretrain model checkpoints/STDCNet813M_73.91.tar
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)
not supported optimizer, using adam



epoch 2, lr 0.009639:   4%|▍         | 60/1570 [00:31<11:43,  2.15it/s, loss=0.000077]

KeyboardInterrupt: ignored